## Dataset Relationships

- companies.company_uuid → rounds.company_uuid
- rounds.funding_round_uuid → investments.funding_round_uuid
- companies.company_uuid → acquisitions.acquiree_uuid

These keys will be used for SQL joins and analysis.


In [1]:
%pip install duckdb


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
    --------------------------------------- 0.3/12.3 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.3 MB 17.1 MB/s eta 0:00:01
   ------ --------------------------------- 2.1/12.3 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.3 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.3 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.3 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.3 MB 4.2 MB/s eta 0:00:03
   ---------- ----------------------------- 3.1/12.3 MB 2.0 MB/s eta 0:00:05
   ------------- -------------------------- 4.2/12.3 MB 2.3 MB/s eta 0:00:04
   ------------- -------------------------- 4.2/12.3 MB 2.3 MB/s eta 0:00:04
   ----------------- ---------------------- 5.2/12.3 MB 2.5 MB/s eta 0:00:03
   --------

In [ ]:
import duckdb
duckdb.__version__

'1.4.3'

In [2]:
import duckdb

con = duckdb.connect("crunchbase.duckdb")
con

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE companies AS
SELECT * FROM read_csv_auto(
    'C:/Users/jayad/Documents/Datasets/Crunchbase1/cleaned/companies_cleaned.csv'
)
""")

con.execute("""
CREATE OR REPLACE TABLE rounds AS
SELECT * FROM read_csv_auto(
    'C:/Users/jayad/Documents/Datasets/Crunchbase1/cleaned/rounds_cleaned.csv'
)
""")

con.execute("""
CREATE OR REPLACE TABLE investments AS
SELECT * FROM read_csv_auto(
    'C:/Users/jayad/Documents/Datasets/Crunchbase1/cleaned/investments_cleaned.csv'
)
""")

con.execute("""
CREATE OR REPLACE TABLE acquisitions AS
SELECT * FROM read_csv_auto(
    'C:/Users/jayad/Documents/Datasets/Crunchbase1/cleaned/acquisitions_cleaned.csv'
)
""")



In [5]:
con.execute("SHOW TABLES").df()

,name
0,acquisitions
1,companies
2,investments
3,rounds


In [6]:
con.execute("""
SELECT
  'companies' AS table, COUNT(*) AS rows FROM companies
UNION ALL
SELECT 'rounds', COUNT(*) FROM rounds
UNION ALL
SELECT 'investments', COUNT(*) FROM investments
UNION ALL
SELECT 'acquisitions', COUNT(*) FROM acquisitions
""").df()


,table,rows
0,companies,17727
1,rounds,31679
2,investments,52870
3,acquisitions,6185


In [2]:
con.execute("""
SELECT
    c.name AS company_name,
    SUM(r.raised_amount_usd) AS total_funding_usd
FROM companies c
JOIN rounds r
    ON c.company_uuid = r.company_uuid
WHERE r.raised_amount_usd IS NOT NULL
GROUP BY c.name
ORDER BY total_funding_usd DESC
LIMIT 10
""").df()


NameError: name 'con' is not defined

To create the connection with DuckDB:

In [3]:
import duckdb

con = duckdb.connect("crunchbase.duckdb")


In [4]:
con.execute("SHOW TABLES").df()


,name
0,acquisitions
1,companies
2,investments
3,rounds


TOP FUNDING COMPANIES: 'below

In [6]:
con.execute("""
SELECT
    c.name AS company_name,
    SUM(r.raised_amount_usd) AS total_funding_usd
FROM companies c
JOIN rounds r
    ON c.permalink = r.company_permalink
WHERE r.raised_amount_usd IS NOT NULL
GROUP BY c.name
ORDER BY total_funding_usd DESC
LIMIT 10
""").df()


,company_name,total_funding_usd
0,Clearwire,5.700000e+09
1,Verizon,3.985050e+09
2,sigmacare,2.600000e+09
3,Facebook,2.425700e+09
4,Carestream,2.400000e+09
5,Solyndra,1.653860e+09
6,Fisker Automotive,1.451000e+09
7,Comcast,1.275000e+09
8,Terra-Gen Power,1.200000e+09
9,Twitter,1.160167e+09


In [8]:
con.execute("DESCRIBE investments").df()


,column_name,column_type,null,key,default,extra
0,company_permalink,VARCHAR,YES,None,None,None
1,company_name,VARCHAR,YES,None,None,None
2,company_category_code,VARCHAR,YES,None,None,None
3,company_country_code,VARCHAR,YES,None,None,None
4,company_state_code,VARCHAR,YES,None,None,None
5,company_region,VARCHAR,YES,None,None,None
6,company_city,VARCHAR,YES,None,None,None
7,investor_permalink,VARCHAR,YES,None,None,None
8,investor_name,VARCHAR,YES,None,None,None
9,investor_category_code,VARCHAR,YES,None,None,None


✅ FIXED QUERY: Investor activity (WORKING)
✔ Count total investments per investor

In [9]:
con.execute("""
SELECT
    investor_name,
    COUNT(*) AS investments_made
FROM investments
WHERE investor_name IS NOT NULL
GROUP BY investor_name
ORDER BY investments_made DESC
LIMIT 10
""").df()


,investor_name,investments_made
0,New Enterprise Associates,445
1,SV Angel,436
2,Kleiner Perkins Caufield & Byers,393
3,Sequoia Capital,369
4,Draper Fisher Jurvetson (DFJ),360
5,Intel Capital,331
6,First Round Capital,326
7,Accel Partners,322
8,Techstars,267
9,500 Startups,254


(more accurate): Avoid double-counting same company

In [10]:
con.execute("""
SELECT
    investor_name,
    COUNT(DISTINCT company_permalink) AS companies_invested_in
FROM investments
WHERE investor_name IS NOT NULL
GROUP BY investor_name
ORDER BY companies_invested_in DESC
LIMIT 10
""").df()


,investor_name,companies_invested_in
0,SV Angel,399
1,New Enterprise Associates,283
2,Techstars,241
3,Intel Capital,228
4,Kleiner Perkins Caufield & Byers,225
5,500 Startups,221
6,Sequoia Capital,215
7,Draper Fisher Jurvetson (DFJ),206
8,Accel Partners,186
9,First Round Capital,186


Why this is actually GOOD for your portfolio

You can now honestly say:

“The investments dataset did not contain a funding-round identifier,
so I adjusted the aggregation logic to count investment events and
distinct portfolio companies instead.”

That shows:

✔ Schema awareness

✔ Analytical judgment

✔ Adaptability to messy data

## Investor Activity Metric

The investments dataset does not include a unique funding round identifier.
Each record represents an individual investment event, so investor activity
is measured using total investment count and distinct portfolio companies.


Investor influence vs funding size:

Question: Do investors who invest more often tend to be involved in larger funding amounts?

In [12]:
con.execute("""
SELECT
    i.investor_name,
    COUNT(*) AS investments_made,
    SUM(r.raised_amount_usd) AS total_funding_usd
FROM investments i
JOIN rounds r
    ON i.company_permalink = r.company_permalink
WHERE i.investor_name IS NOT NULL
  AND r.raised_amount_usd IS NOT NULL
GROUP BY i.investor_name
ORDER BY total_funding_usd DESC
LIMIT 10
""").df()


,investor_name,investments_made,total_funding_usd
0,Kleiner Perkins Caufield & Byers,1383,4.195685e+10
1,New Enterprise Associates,1406,3.229488e+10
2,Draper Fisher Jurvetson (DFJ),1298,2.195388e+10
3,Accel Partners,993,2.113344e+10
4,Sequoia Capital,1233,2.104540e+10
5,Intel,50,1.782197e+10
6,Time Warner,42,1.747420e+10
7,Comcast,28,1.721200e+10
8,Sprint Nextel,12,1.710000e+10
9,Greylock Partners,765,1.660617e+10


Insights:
Highly active investors tend to be associated with larger total funding amounts,
suggesting that frequent investors often participate in high-value funding rounds.


Funding over time (year & quarter trends):

Question: How has startup funding changed over time?

In [13]:
##Year-wise Funding:
con.execute("""
SELECT
    funded_year,
    SUM(raised_amount_usd) AS total_funding_usd
FROM rounds
WHERE funded_year IS NOT NULL
  AND raised_amount_usd IS NOT NULL
GROUP BY funded_year
ORDER BY funded_year
""").df()


,funded_year,total_funding_usd
0,1960,5.180000e+07
1,1984,1.000000e+05
2,1987,2.500000e+06
3,1989,1.500000e+04
4,1990,1.000000e+06
5,1993,1.250000e+05
6,1995,1.530000e+07
7,1996,1.612500e+06
8,1997,5.700000e+07
9,1998,1.235370e+08


In [14]:
##Quater-wise funding:
con.execute("""
SELECT
    funded_quarter,
    SUM(raised_amount_usd) AS total_funding_usd
FROM rounds
WHERE funded_quarter IS NOT NULL
  AND raised_amount_usd IS NOT NULL
GROUP BY funded_quarter
ORDER BY funded_quarter
""").df()


,funded_quarter,total_funding_usd
0,1960-Q1,5.180000e+07
1,1984-Q1,1.000000e+05
2,1987-Q1,2.500000e+06
3,1989-Q1,1.500000e+04
4,1990-Q1,1.000000e+06
...,...,...
74,2012-Q4,9.842475e+09
75,2013-Q1,1.200951e+10
76,2013-Q2,1.762274e+10
77,2013-Q3,1.911029e+10


Insight: Funding activity shows clear temporal trends, with certain years and quarters
attracting significantly higher investment volumes.


Acquisition likelihood vs funding (VERY strong portfolio section)

Question: Do companies that acquire others tend to have higher funding?

In [16]:
con.execute("""
SELECT
    CASE
        WHEN a.acquirer_permalink IS NOT NULL THEN 'Acquirer'
        ELSE 'Non-Acquirer'
    END AS acquisition_status,
    AVG(r.raised_amount_usd) AS avg_funding_usd,
    COUNT(DISTINCT c.permalink) AS company_count
FROM companies c
LEFT JOIN acquisitions a
    ON c.permalink = a.acquirer_permalink
LEFT JOIN rounds r
    ON c.permalink = r.company_permalink
WHERE r.raised_amount_usd IS NOT NULL
GROUP BY acquisition_status
""").df()


,acquisition_status,avg_funding_usd,company_count
0,Non-Acquirer,8.351903e+06,15471
1,Acquirer,5.957094e+07,710


Insights: Companies that act as acquirers tend to have higher average funding levels
compared to non-acquiring companies, suggesting that access to capital
enables inorganic growth through acquisitions.
